In [1]:
#V1版本的不足之处;不够智能，还需要自己设定不同二维材料种类
import numpy as np
import pandas as pd
import random
import os
from shutil import copy
from math import cos, sin, atan2, sqrt, pi ,radians, degrees

In [2]:
# 打开新文件
def open_poscar(name):
    f = open(name,'r+')
    flist= f.readlines()
    f.close()
    return flist

In [3]:
# 分割结构层数
def countlayers(start_num, super_cell, line, sample_type):
    if sample_type == 'MAZ':
        thresthold = 0.01
        num_atoms = 7*super_cell[0]*super_cell[1]
        layer_num = 7
    x_cartesian = []
    y_cartesian = []
    z_cartesian = []
#     a1 = float(line[2].split()[0])
#     a2 = float(line[3].split()[0])
#     a3 = float(line[4].split()[0])
#     b1 = float(line[2].split()[1])
#     b2 = float(line[3].split()[1])
#     b3 = float(line[4].split()[1])
#     z1 = float(line[2].split()[2])
#     z2 = float(line[3].split()[2])
#     z3 = float(line[4].split()[2])
#     if  line[8].strip().upper().startswith('C') or line[7].strip().upper().startswith('C'):
#         a2 = a3 = b1 = b3 = z1 = z2  = 0
#         a1 = b2 = z3 = 1
    for i in range(start_num, num_atoms + start_num): 
#         x_cartesian.append(float(line[i].split()[0]) * a1 + float(line[i].split()[1]) * a2 + float(line[i].split()[2]) * a3)
#         y_cartesian.append(float(line[i].split()[0]) * b1 + float(line[i].split()[1]) * b2 + float(line[i].split()[2]) * b3)
#         z_cartesian.append(float(line[i].split()[0]) * z1 + float(line[i].split()[1]) * z2 + float(line[i].split()[2]) * z3)
        x_cartesian.append(float(line[i].split()[0]))
        y_cartesian.append(float(line[i].split()[1]))
        z_cartesian.append(float(line[i].split()[2]))                
        layerscount_initial = determinelayers(z_cartesian, thresthold)
    if len(layerscount_initial) == layer_num:
        layerscount = layerscount_initial
    elif len(layerscount_initial) > layer_num:
        while len(layerscount_initial) > layer_num:
            thresthold = thresthold + 0.001
            layerscount_initial = determinelayers(z_cartesian, thresthold)
            if len(layerscount_initial) == layer_num:
                layerscount = layerscount_initial
                break
    elif len(layerscount_initial) < layer_num:
        while len(layerscount_initial) < layer_num:
            thresthold = thresthold - 0.001
            layerscount_initial = determinelayers(z_cartesian, thresthold)
            if len(layerscount_initial) == layer_num:
                layerscount = layerscount_initial
                break
    return layerscount_initial, x_cartesian, y_cartesian, z_cartesian

In [4]:
# 确定层数
def determinelayers(z_cartesian, thresthold):
    seq = sorted(z_cartesian)
    min = seq[0]
    layerscount = {}
    sets = [min]
    for j in range(len(seq)):
        if abs(seq[j]-min) >= thresthold:
            min = seq[j]
            sets.append(min)
    for i in range(1,len(sets)+1):
        layerscount[i] = []            
        if i > 1:
            for k in range(len(z_cartesian)):   
                if abs(z_cartesian[k]-sets[i-1]) <= thresthold and not abs(z_cartesian[k]-sets[i-2]) <= thresthold:
                    layerscount[i].append(k)            
        else:
            for k in range(len(z_cartesian)):   
                if abs(z_cartesian[k]-sets[i-1]) <= thresthold:
                    layerscount[i].append(k)
    return layerscount

In [5]:
#生成负载结构
def gernerate_SupStructure(line, layerscount, x_cartesian, y_cartesian, z_cartesian, 
                           sample_type, ads_type):
    if sample_type == 'MAZ':
        if ads_type == 'top':
            top_list = layerscount[7]
            center_index = get_support_index(top_list, x_cartesian, y_cartesian, z_cartesian,ads_type)
        elif ads_type == 'fcc':
            fcc_list = layerscount[6]
            center_index = get_support_index(fcc_list, x_cartesian, y_cartesian, z_cartesian,ads_type)
        elif ads_type == 'hcp':
            hcp_list = layerscount[4]
            center_index = get_support_index(hcp_list, x_cartesian, y_cartesian, z_cartesian,ads_type)
    return center_index

In [6]:
# 列表索引
def list_indice(x,y):
    return [x[i] for i in y] #x为需要索引的列表，y为索引列表

In [7]:
#确定中心原子序号
def get_support_index(sup_layer, x_cartesian, y_cartesian, z_cartesian,ads_type):
    x_cartesian_sup = list_indice(x_cartesian, sup_layer)
    y_cartesian_sup = list_indice(y_cartesian, sup_layer)
    z_cartesian_sup = list_indice(z_cartesian, sup_layer)
    position_xy = list(zip(x_cartesian_sup, y_cartesian_sup))
    position_center = list(center_geolocation(position_xy))
    position_center.append(max(z_cartesian_sup))
    min_index = get_replace(position_center, x_cartesian_sup, y_cartesian_sup, z_cartesian_sup)
    replace_atom_index = sup_layer[min_index]
    return replace_atom_index

In [8]:
# 确定掺杂原子与替换原子的最短距离
def get_replace(position_center, x_cartesian_top, y_cartesian_top, z_cartesian_top):
    position_xyz = list(zip(x_cartesian_top, y_cartesian_top, z_cartesian_top))
    min_distance = float('inf')
    min_index = []
    for i in range(len(position_xyz)):
        distance = sqrt((position_center[0]-position_xyz[i][0])**2 + (position_center[1]-position_xyz[i][1])**2  
                        + (position_center[2]-position_xyz[i][2])**2)
        if distance < min_distance:
            min_distance = distance
            min_index = i    
    return min_index

In [9]:
# 确定最上层原子中心
def center_geolocation(geolocations):
    x = 0
    y = 0
    z = 0
    lenth = len(geolocations)
    for lon, lat in geolocations:
        lon = radians(float(lon))
        lat = radians(float(lat))
        x += cos(lat) * cos(lon)
        y += cos(lat) * sin(lon)
        z += sin(lat)    
    x = float(x / lenth)
    y = float(y / lenth)
    z = float(z / lenth)
    center = (degrees(atan2(y, x)), degrees(atan2(z, sqrt(x * x + y * y))))    
    return center

In [22]:
#生成负载型单原子金属MAZ的POSCAR
def get_support_poscar(line, center_index, x_cartesian, y_cartesian, z_cartesian, 
                       sample_type, ads_type, MAZ_name):
    if sample_type == 'MAZ':
        if ads_type == 'top':
            top_list = layerscount[7]
            center_index = get_support_index(top_list, x_cartesian, y_cartesian, z_cartesian,ads_type)
            new_poscar = get_newposcar_data(flist, x_cartesian, y_cartesian, z_cartesian, 
                       center_index, sample_type, ads_type, MAZ_name)
        elif ads_type == 'fcc':
            fcc_list = layerscount[6]
            center_index = get_support_index(fcc_list, x_cartesian, y_cartesian, z_cartesian,ads_type)
            new_poscar = get_newposcar_data(flist,x_cartesian, y_cartesian, z_cartesian, 
                       center_index, sample_type, ads_type, MAZ_name)
        elif ads_type == 'hcp':
            hcp_list = layerscount[4]
            center_index = get_support_index(hcp_list, x_cartesian, y_cartesian, z_cartesian,ads_type)
            new_poscar = get_newposcar_data(flist,x_cartesian, y_cartesian, z_cartesian, 
                       center_index, sample_type, ads_type, MAZ_name)
    return new_poscar

In [23]:
# 组成新文件数据
def get_newposcar_data(flist,x_cartesian, y_cartesian, z_cartesian, 
                       center_index, sample_type, ads_type, MAZ_name):
    Z_name = MAZ_name.split('_')[-1]
    line1 = [str(x) for x in flist[0].split()]
    line2 = [str(x) for x in flist[1].split()]
    line3 = [str(x) for x in flist[2].split()]
    line4 = [str(x) for x in flist[3].split()]
    line5 = [str(x) for x in flist[4].split()]
    line6 = [str(x) for x in flist[5].split()] + ['H'] # 原子序号
    line7 = [str(x) for x in flist[6].split()] + ['1'] #原子个数
    line8 = [str(x) for x in flist[7].split()]
    new_list = []
    new_list.append(line1)
    new_list.append(line2)
    new_list.append(line3)
    new_list.append(line4)
    new_list.append(line5)
    new_list.append(line6)
    new_list.append(line7)
    new_list.append(line8)
    if sample_type == 'MAZ':
        radii_sum = get_radii_sum('H', Z_name)
        c = float(flist[4].split()[-1])
        cartesian_xyz_bare = list(zip(x_cartesian, y_cartesian, z_cartesian))
        if ads_type == 'top':
            cartesian_xyz_top = [(x_cartesian[center_index], y_cartesian[center_index]
                                     , max(z_cartesian)+radii_sum/c)]
            cartesian_xyz = cartesian_xyz_bare + cartesian_xyz_top
        elif ads_type == 'fcc':
            cartesian_xyz_fcc = [(x_cartesian[center_index], y_cartesian[center_index]
                                     , max(z_cartesian)+radii_sum*sqrt(2)/(2*c))]
            cartesian_xyz = cartesian_xyz_bare + cartesian_xyz_fcc
        elif ads_type == 'hcp':
            cartesian_xyz_hcp = [(x_cartesian[center_index], y_cartesian[center_index]
                                     , max(z_cartesian)+radii_sum*sqrt(2)/(2*c))]
            cartesian_xyz = cartesian_xyz_bare + cartesian_xyz_hcp 
    for i in range(len(cartesian_xyz)):
        new_list.append(cartesian_xyz[i])
    return new_list

In [12]:
# 获取原子半径和
def get_radii_sum(atom1, atom2):
    # 获取涉及到的原子半径
    data = pd.read_excel('共价半径.xlsx', header=None)
    # 生成原子半径字典
    data_radius = dict_radius_production(data)
    radii_sum = data_radius[atom1] + data_radius[atom2]
    return radii_sum

In [13]:
# 生成共价半径字典
def dict_radius_production(data):
    dict_radius = {}
    for i in range(len(data)):
        dict_radius.update({data.iloc[i,0]:data.iloc[i,1]})
    return dict_radius

In [14]:
# 写入新文件
def write_newposcar(path, element_name, new_poscar, ads_type):
    new_poscar_name = path + '/POSCAR_' + ''.join(element_name)+ '_' + ads_type
    f2=open(new_poscar_name,"w")
    for i in range(len(new_poscar)):
        s1 = str(new_poscar[i]).replace('[','').replace(']','').replace('(','').replace(')','')
        s2 = s1.replace("'",'').replace(',','') + '\n' 
        f2.write(s2)
    f2.close()

In [15]:
# 获取需要负载MAZ的SMAZ列表
name = '20220822下选择的100个MAZ'
BMAZ_data = pd.read_excel('../I_select_sample/' + name +  '.xlsx', header=None)
BMAZ_list = BMAZ_data.values.flatten().tolist()[1:]

In [24]:
#创建SMAZ的文件夹及放入MAZ的CONTCAR以及SMZA的三种POSCAR
MAZ_structure_list = os.listdir('../../MAZ_structures')
noexist_in_MAZ =[]
for a in BMAZ_list:
    os.makedirs(name+'/'+ a)
    if 'CONTCAR_' + a not in MAZ_structure_list:
        noexist_in_MAZ.append(a)
    for i in MAZ_structure_list:       
        if a == i.split('CONTCAR_')[-1]:
            #复制MAZ的CONTCAR到各样本路径底下
            from_path = '../../MAZ_structures/' + i
            to_path = name + '/'+ a
            copy(from_path, to_path)
noexist_in_MAZ

[]

In [25]:
#参数
Sample_type = 'MAZ' #样本统称
Super_cell = [3,3]  #超胞扩胞数
Start_num = 9       #坐标起始行

In [26]:
#创建SMAZ的文件夹及放入MAZ的CONTCAR以及SMZA的三种POSCAR
layer_num =[]
for a in BMAZ_list:
    MAZ_name =  a
    Z_name = a.split('_')[-1]
    #复制MAZ的CONTCAR到各样本路径底下
    to_path = name +'/'+ a
    #获取MAZ的CONTCAR坐标数据并分层
    flist = open_poscar(to_path+'/CONTCAR_'+a)
    layerscount, x_cartesian, y_cartesian, z_cartesian = countlayers(Start_num-1, Super_cell, flist, Sample_type)
     #创建top，fcc，hcp文件夹并放入POSCAR
    #top位
    top_path = name+'/'+ a +'/top'
    os.makedirs(top_path) 
    top_id = gernerate_SupStructure(flist, layerscount, x_cartesian, y_cartesian, z_cartesian, 
                   Sample_type, 'top')
    top_poscar = get_support_poscar(flist, top_id, x_cartesian, y_cartesian, z_cartesian, 
               Sample_type, 'top', MAZ_name)
    write_newposcar(top_path, MAZ_name, top_poscar, 'top')           
    #fcc位
    fcc_path = name+'/'+ a +'/fcc'
    os.makedirs(fcc_path) 
    fcc_id = gernerate_SupStructure(flist, layerscount, x_cartesian, y_cartesian, z_cartesian,  
                   Sample_type, 'fcc')
    fcc_poscar = get_support_poscar(flist, fcc_id, x_cartesian, y_cartesian, z_cartesian, 
               Sample_type, 'fcc', MAZ_name)
    write_newposcar(fcc_path, MAZ_name, fcc_poscar, 'fcc')           
    #hcp位
    hcp_path = name+'/'+ a +'/hcp'
    os.makedirs(hcp_path) 
    hcp_id = gernerate_SupStructure(flist, layerscount, x_cartesian, y_cartesian, z_cartesian,  
                   Sample_type, 'hcp')   
    hcp_poscar = get_support_poscar(flist, hcp_id, x_cartesian, y_cartesian, z_cartesian, 
               Sample_type, 'hcp', MAZ_name)
    write_newposcar(hcp_path, MAZ_name, hcp_poscar, 'hcp')